In [2]:
# 4/8/23
# KM Altenburger

# steps for downloading repo internally; run w/ "grasp_workflows kernel"
# sudo feature install ttls_fwdproxy
# mgt import --src-type pypi sdist
# mgt import --src-type pypi nvidia-cublas-cu11==11.10.3.66
# mgt import --src-type pypi ogb

import sys
sys.path.insert(0, "/home/kaltenburger/fbsource/ogb")
import ogb
import torch
#import torch_geometric
from ogb.nodeproppred import NodePropPredDataset
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_auc_score, precision_recall_curve, auc)
from sklearn.model_selection import train_test_split


Bad key "axes.color_cycle" on line 214 in
/home/kaltenburger/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


### read & set-up dataset


In [3]:
dataset = NodePropPredDataset(name='ogbn-proteins')
g = dataset.graph # get edgelist and graph metadata
y = dataset.labels # outcomes
node_species_df = pd.DataFrame({'node_id':range(len(g['node_species'].flatten())), 'node_species': g['node_species'].flatten()})
node_species_df.head()

,node_id,node_species
0,0,3702
1,1,3702
2,2,3702
3,3,3702
4,4,3702


In [4]:
# create edgelist
edge_list_df = pd.DataFrame(g["edge_index"].T)
edge_list_df.columns = ['src','dst']
edge_list_df.head()

,src,dst
0,1,96401
1,96401,1
2,1,68343
3,68343,1
4,1,125069


In [5]:
# sanity check on inter-species connections
edge_list_df_check = edge_list_df.merge(node_species_df, how = 'left', left_on = 'src', right_on = 'node_id')
edge_list_df_check.drop('node_id', axis = 1, inplace=True)
edge_list_df_check = edge_list_df_check.merge(node_species_df, how = 'left', left_on = 'dst', right_on = 'node_id')
edge_list_df_check.drop('node_id', axis = 1, inplace=True)
#edge_list_df_check.head()
print(np.mean(edge_list_df_check['node_species_x']==edge_list_df_check['node_species_y']))
print(np.mean(edge_list_df_check['node_species_x']!=edge_list_df_check['node_species_y']))

0.8059610954678583


0.1940389045321417


In [6]:
y0 = pd.DataFrame(y)
#y0 = y0.iloc[:,0:25] # part I
#y0 = y0.iloc[:,25:50] # part II
y0 = y0.iloc[:,50:112] # part II
y0['node_id'] = range(len(y0[50]))
    
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

X_train = y0['node_id'].iloc[train_idx]
y_train = y0.iloc[train_idx]

X_valid = y0['node_id'].iloc[valid_idx]
X_test = y0['node_id'].iloc[test_idx]
y_valid = y0.iloc[valid_idx]
y_test = y0.iloc[test_idx]



In [7]:
edge_w = pd.DataFrame(g['edge_feat'])
edge_w = edge_w.add_suffix('_weight')
edge_list_df = pd.concat([edge_list_df, edge_w], axis=1)
#edge_list_df.head()
# w0 = edge_w[j]
# edge_list_df['weight'] = w0
# edge_list_df.head()
# # merge in labels for training nodes on a user's connections
relevant_edges_tmp = edge_list_df.merge(y_train, left_on = 'dst', right_on='node_id', how = 'left')
relevant_edges_tmp.head()

,src,dst,0_weight,1_weight,2_weight,3_weight,4_weight,5_weight,6_weight,7_weight,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,node_id
0,1,96401,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,96401,1,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,68343,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68343.0
3,68343,1,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,125069,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
from sklearn.ensemble import RandomForestClassifier

# set-up loop
rocauc_v0_ud_w = []
#rocauc_v2_ud_w = []
#rocauc_ud_uw = []

baseline_prop = []

#for i in range(0,25): # loop through outcomes
#for i in range(25,50):
for i in range(50,112):
    print(i)
#i = 0 # testing only right now

    two_hop_v0_dict = {}
    for j in range(8): # loop through edge weights
        #print(j)

        # edge weights
        # edge_w = pd.DataFrame(g['edge_feat'])
        # #w0 = np.mean(edge_w,1) # average edge weight
        # w0 = edge_w[j]
        # edge_list_df['weight'] = w0
        # edge_list_df.head()
        # # merge in labels for training nodes on a user's connections
        # relevant_edges_tmp = edge_list_df.merge(y_train, left_on = 'dst', right_on='node_id', how = 'left')
        relevant_edges = relevant_edges_tmp[["src", "dst", str(j)+"_weight", "node_id", i]]
        relevant_edges.columns = ["src", "dst", "weight", "node_id", "y"]
        label1 = relevant_edges.groupby(["src"])["y"].sum().reset_index()
        labeltotal = relevant_edges.groupby(["src"])["y"].count().reset_index()
        relevant_edgelist = labeltotal.merge(label1, on="src")
        relevant_edgelist["label_0"] = relevant_edgelist["y_x"] - relevant_edgelist["y_y"]
        relevant_edgelist.rename(columns={"y_y": "label_1"}, inplace=True)
        relevant_edgelist.drop(["y_x"], axis=1, inplace=True)

        ## v0 - linear combination of single layers
        normalizer = pd.DataFrame(y_train[i].value_counts(normalize=True, ascending=True))

        # step 1 - one hop scores
        step1_one_hop_scores = pd.DataFrame(
            {
                "src": relevant_edgelist.src,
                "frac": np.log(
                    (relevant_edgelist["label_1"] + 1) / (relevant_edgelist["label_0"] + 1)
                )
                * list((normalizer.iloc[0]) / (normalizer.iloc[1]))[0],
            }
        )

        # step 2 - two hop scores
        two_hop_stg = relevant_edges.merge(
            step1_one_hop_scores, left_on="dst", right_on="src", how="inner"
        )

        #print(np.shape(two_hop_stg))
        interval = pd.DataFrame(
            two_hop_stg.weight.quantile([0.25, 0.5, 0.75])
        ).reset_index()
        interval.columns = ["interval", "weight"]
        #print(interval)
        #print(np.unique(interval.weight))
        if len(np.unique(interval.weight)) ==1:
            interval = pd.DataFrame(
                two_hop_stg.weight.quantile([0.5])
            ).reset_index()
            interval.columns = ["interval", "weight"]
            
            two_0 = (
                two_hop_stg[two_hop_stg.weight <= interval.weight[0]]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )
            
            two_1 = (
                two_hop_stg[(two_hop_stg.weight > interval.weight[0])]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )

            two_hop_v0 = two_0.merge(two_1, how="outer", on="src_x").fillna(0)
            two_hop_v0.columns = ["src_x", "frac1_"+str(j), "frac2_"+str(j), ]

        if len(np.unique(interval.weight)) ==2:
            interval = pd.DataFrame(
                two_hop_stg.weight.quantile([0.25, 0.75])
            ).reset_index()
            interval.columns = ["interval", "weight"]
            two_0 = (
                two_hop_stg[two_hop_stg.weight <= interval.weight[0]]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )
            two_1 = (
                two_hop_stg[
                    (two_hop_stg.weight > interval.weight[0])
                    & (two_hop_stg.weight <= interval.weight[1])
                ]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )
            two_2 = (
                two_hop_stg[
                    (two_hop_stg.weight > interval.weight[1])
                    
                ]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )

        else:
            interval = pd.DataFrame(
                two_hop_stg.weight.quantile([0.25, 0.5, 0.75])
            ).reset_index()
            interval.columns = ["interval", "weight"]

            two_0 = (
                two_hop_stg[two_hop_stg.weight <= interval.weight[0]]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )
            two_1 = (
                two_hop_stg[
                    (two_hop_stg.weight > interval.weight[0])
                    & (two_hop_stg.weight <= interval.weight[1])
                ]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )
            two_2 = (
                two_hop_stg[
                    (two_hop_stg.weight > interval.weight[1])
                    & (two_hop_stg.weight <= interval.weight[2])
                ]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )
            two_3 = (
                two_hop_stg[(two_hop_stg.weight > interval.weight[2])]
                .groupby(["src_x"])["frac"]
                .sum()
                .reset_index()
            )

            two_hop_v0 = two_0.merge(two_1, how="outer", on="src_x").fillna(0)
            two_hop_v0 = two_hop_v0.merge(two_2, how="outer", on="src_x").fillna(0)
            two_hop_v0 = two_hop_v0.merge(two_3, how="outer", on="src_x").fillna(0)
            two_hop_v0.columns = ["src_x", "frac1_"+str(j), "frac2_"+str(j), "frac3_"+str(j), "frac4_"+str(j)]
        # two_hop_v0_valid = two_hop_v0.merge(
        #     y_valid, left_on="src_x", right_on="node_id", how="right"
        # )

        two_hop_v0.head()
        two_hop_v0_dict[j] = two_hop_v0

    # rename columns -- do multiple joins
    two_hop_v0_merge = two_hop_v0_dict[0].merge(two_hop_v0_dict[1], on = 'src_x', how = 'outer')
    two_hop_v0_valid = two_hop_v0_merge.merge(
            y_valid, left_on="src_x", right_on="node_id", how="right"
        )

    two_hop_v0_merge = two_hop_v0_dict[0].merge(two_hop_v0_dict[1], on = 'src_x', how = 'outer')
    for k in range(2,7):
        #print(k)
        two_hop_v0_merge = two_hop_v0_merge.merge(two_hop_v0_dict[k+1], on = 'src_x', how = 'outer')



    two_hop_v0_valid = two_hop_v0_merge.merge(
            y_valid, left_on="src_x", right_on="node_id", how="right"
        )
    clf = LogisticRegression(random_state=0).fit(
        two_hop_v0_valid[['frac1_0',   'frac2_0',   'frac3_0',   'frac4_0',
            'frac1_1',   'frac2_1',   'frac3_1',   'frac4_1', 'frac1_3_x',
        'frac2_3_x', 'frac3_3_x', 'frac4_3_x', 'frac1_3_y', 'frac2_3_y',
        'frac3_3_y', 'frac4_3_y',   'frac1_3',   'frac2_3',   'frac3_3',
            'frac4_3',   'frac1_6',   'frac2_6',   'frac3_6',   'frac4_6',
            'frac1_7',   'frac2_7',   'frac3_7',   'frac4_7']], two_hop_v0_valid[i]
    )


    pred_v0 = y_test.merge(two_hop_v0_merge, right_on="src_x", left_on="node_id", how="left")
    print(
        "roc-auc, v1: ",
        roc_auc_score(
            pred_v0[i],
            clf.predict_proba(pred_v0[['frac1_0',   'frac2_0',   'frac3_0',   'frac4_0',
            'frac1_1',   'frac2_1',   'frac3_1',   'frac4_1', 'frac1_3_x',
        'frac2_3_x', 'frac3_3_x', 'frac4_3_x', 'frac1_3_y', 'frac2_3_y',
        'frac3_3_y', 'frac4_3_y',   'frac1_3',   'frac2_3',   'frac3_3',
            'frac4_3',   'frac1_6',   'frac2_6',   'frac3_6',   'frac4_6',
            'frac1_7',   'frac2_7',   'frac3_7',   'frac4_7']])[:,1]
        ),
    )


    print('baseline, v1: ', np.mean(pred_v0[i]), len(pred_v0[i]))

    rocauc_v0_ud_w.append(
        roc_auc_score(
            pred_v0[i],
            clf.predict_proba(pred_v0[['frac1_0',   'frac2_0',   'frac3_0',   'frac4_0',
            'frac1_1',   'frac2_1',   'frac3_1',   'frac4_1', 'frac1_3_x',
        'frac2_3_x', 'frac3_3_x', 'frac4_3_x', 'frac1_3_y', 'frac2_3_y',
        'frac3_3_y', 'frac4_3_y',   'frac1_3',   'frac2_3',   'frac3_3',
            'frac4_3',   'frac1_6',   'frac2_6',   'frac3_6',   'frac4_6',
            'frac1_7',   'frac2_7',   'frac3_7',   'frac4_7']])[:,1]
        ),
    )





## v2 undirected, weighted
# W = 4
# interval = pd.DataFrame(relevant_edges.weight.quantile([0.25,0.5,0.75])).reset_index()
# interval.columns = ['interval', 'weight']

# relevant_edgelist_dict = {}
# for w in range(W):
#     if w == 0:
#         idx = relevant_edges.weight<=interval.weight[0]
#     if w == 1:
#         idx = (relevant_edges.weight>interval.weight[0]) & (relevant_edges.weight<=interval.weight[1])
#     if w == 2:
#         idx = (relevant_edges.weight>interval.weight[1]) & (relevant_edges.weight<=interval.weight[2])
#     if w == 3:
#         idx = (relevant_edges.weight>interval.weight[2])

#     label1 = relevant_edges.iloc[np.where(idx)[0]].groupby(['src'])['y'].sum().reset_index()
#     labeltotal = relevant_edges.iloc[np.where(idx)[0]].groupby(['src'])['y'].count().reset_index()

#     relevant_edgelist = labeltotal.merge(label1, on = 'src', how='outer').fillna(0)
#     relevant_edgelist['label_0']  = relevant_edgelist['y_x'] - relevant_edgelist['y_y']
#     relevant_edgelist.rename(columns = {'y_y':'label_1'}, inplace = True)
#     relevant_edgelist.drop(['y_x'], axis=1,inplace = True)
#     relevant_edgelist.columns = ['src', 'label_1_' + str(w),'label_0_' + str(w)]
#     relevant_edgelist_dict[w]=relevant_edgelist

# merged_edgelist = relevant_edgelist_dict[0].merge(relevant_edgelist_dict[1], on = 'src', how = 'outer')
# merged_edgelist = merged_edgelist.merge(relevant_edgelist_dict[2], on = 'src', how = 'outer')
# merged_edgelist = merged_edgelist.merge(relevant_edgelist_dict[3], on = 'src', how = 'outer').fillna(0)
# normalizer = pd.DataFrame(y_train[i].value_counts(normalize=True, ascending=True))
# step1_one_hop_scores = pd.DataFrame({'src':merged_edgelist.src, 'frac0':np.log((merged_edgelist['label_1_0']+1)/(merged_edgelist['label_0_0']+1)) * list((normalizer.iloc[0]) / (normalizer.iloc[1]))[0],
# 'frac1':np.log((merged_edgelist['label_1_1']+1)/(merged_edgelist['label_0_1']+1)) * list((normalizer.iloc[0]/normalizer.iloc[1]))[0],
# 'frac2':np.log((merged_edgelist['label_1_2']+1)/(merged_edgelist['label_0_2']+1)) * list((normalizer.iloc[0]/normalizer.iloc[1]))[0],
# 'frac3':np.log((merged_edgelist['label_1_3']+1)/(merged_edgelist['label_0_3']+1)) * list((normalizer.iloc[0]/normalizer.iloc[1]))[0],

# })
# two_hop_stg1 = relevant_edges.merge(step1_one_hop_scores, left_on = 'dst', right_on='src', how = 'inner')

# two_hop_v2 = two_hop_stg1.groupby(['src_x'])[('frac0','frac1','frac2','frac3')].sum().reset_index()
# two_hop_v2_valid = two_hop_v2.merge(y_valid,left_on = 'src_x', right_on = 'node_id', how = 'right')
# pred_v2 = y_test.merge(two_hop_v2, right_on="src_x", left_on="node_id", how="left")
# clf = LogisticRegression(random_state=0).fit(two_hop_v2_valid[['frac0', 'frac1', 'frac2', 'frac3']], two_hop_v2_valid[i])

# print('roc-auc, v2: ',roc_auc_score(pred_v2[i], clf.predict_proba(pred_v2[["frac0","frac1", "frac2", "frac3"]])[:,1]))
# precision, recall, thresholds = precision_recall_curve(pred_v2[i], clf.predict_proba(pred_v2[["frac0","frac1", "frac2", "frac3" ]])[:,1])
# print('aucpr, v2: ',auc(recall, precision))
# print('baseline, v2: ', np.mean(pred_v2[i]), len(pred_v2[i]))

# aucpr_v2_ud_w.append(auc(recall, precision))
# rocauc_v2_ud_w.append(roc_auc_score(pred_v2[i],clf.predict_proba(pred_v2[["frac0","frac1", "frac2", "frac3"]])[:,1])) 

# ## undirected, unweighted
# two_hop = two_hop_stg.groupby(["src_x"])["frac"].sum().reset_index()
# two_hop.columns = ["src", "frac"]
# two_hop_pred = two_hop.merge(X_test, left_on="src", right_on="node_id", how="right")
# pred = y_test.merge(
#     two_hop_pred,
#     on="node_id",
# )
# print("roc-auc, ud/uw: ", roc_auc_score(pred[i], pred["frac"]))
# rocauc_ud_uw.append(roc_auc_score(pred[i], pred["frac"]))
# precision, recall, thresholds = precision_recall_curve(pred[i], pred["frac"])
# print("aucpr, ud/uw: ", auc(recall, precision))
# print('baseline, ud/uw: ', np.mean(pred[i]), len(pred[i]))

# aucpr_ud_uw.append(auc(recall, precision))
# baseline_prop.append(np.mean(pred[i]))
# print("************************************************************************")

50


roc-auc, v1:  0.6752492300316326
baseline, v1:  0.014060537298918109 24679
51


roc-auc, v1:  0.714391398402834
baseline, v1:  0.024757891324607966 24679
52


roc-auc, v1:  0.6937062947861592
baseline, v1:  0.015114064589326959 24679
53


roc-auc, v1:  0.7603069887674996
baseline, v1:  0.013007010008509259 24679
54


...

roc-auc, v1:  0.6645777686459832
baseline, v1:  0.013938976457717087 24679
108


roc-auc, v1:  0.718259199511065
baseline, v1:  0.008387698042870456 24679
109


roc-auc, v1:  0.6825983139830228
baseline, v1:  0.012277644961303132 24679
110


roc-auc, v1:  0.6979073661107844
baseline, v1:  0.009400705052878966 24679
111


roc-auc, v1:  0.7065668712083706
baseline, v1:  0.00875238056647352 24679


In [9]:
print(np.mean(rocauc_v0_ud_w))


0.7141170451727397


In [10]:
# with open('rocauc_v0_ud_w_part1_multilayer.npy', 'wb') as f:
# with open('rocauc_v0_ud_w_part2_multilayer.npy', 'wb') as f:
with open("rocauc_v0_ud_w_part3_multilayer.npy", "wb") as f:
    np.save(f, rocauc_v0_ud_w)